In [19]:
from influxdb_client import InfluxDBClient
import pandas as pd
token = 'MlUmh8wWKDAcBiZbBjvJPguGKxoON1ZRqFjVQ_zh8M3NHLcr53WnxbfXX_ByD0Wm7S8G0kb622gj2EX25MzFCA=='
org = "Portfolio_Opt"
url = "http://localhost:8080"
bucket = "Nifty50"

client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()


In [22]:
filtered_stocks = ['ASIANPAINT', 'BAJAJ-AUTO', 'HCLTECH', 'INFY', 'TCS', 'TITAN']


In [30]:
for symbol in filtered_stocks:
    query=f'''
    from(bucket: "Nifty50")
        |> range(start: 0)
        |> filter(fn: (r) => r["_measurement"] == "market_data")
        |> filter(fn: (r)=> r["symbol"] == "{symbol}" )
        |> filter(fn: (r) => r["_field"] == "close")
        |> window(every: 1d)
        |> last()
        |> duplicate(column: "_start", as: "_time")
        |> window(every: inf)
        |> yield(name: "closeData")
    '''
    result = query_api.query(org=org, query=query)

    data = []
    for table in result:
        for record in table.records:
            data.append((record.get_time(), record.get_value()))

    df = pd.DataFrame(data, columns=['Date', symbol])
    df.set_index('Date', inplace=True)

    # Combine with the main DataFrame
    if all_data.empty:
        all_data = df
    else:
        all_data = all_data.join(df, how='outer', rsuffix=f'_{symbol}')

pd.DataFrame.to_csv(all_data, 'nifty50.csv')
